In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K


def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


smooth = 1e-15


def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [2]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model


def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def encoder_block(inputs, num_filters):
    s = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(s)
    return s, p


def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x


def build_unet(input_shape):
    """ Input Layer """
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    s5, p5 = encoder_block(p4, 1024)

    """ Bottleneck """
    b1 = conv_block(p5, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s5, 1024)
    d2 = decoder_block(d1, s4, 512)
    d3 = decoder_block(d2, s3, 256)
    d4 = decoder_block(d3, s2, 128)
    d5 = decoder_block(d4, s1, 64)

    """ Output Layer """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d5)

    model = Model(inputs, outputs, name="UNET")

    return model


if __name__ == "__main__":
    model = build_unet((256, 256, 3))
    model.summary()


Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

In [4]:
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from glob import glob
import cv2
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    size = int(len(images) * split)
    train_x, valid_x = train_test_split(
        images, test_size=size, random_state=42)
    train_y, valid_y = train_test_split(
        masks, test_size=size, random_state=42)
    train_x, test_x = train_test_split(
        train_x, test_size=size, random_state=42)
    train_y, test_y = train_test_split(
        train_y, test_size=size, random_state=42)
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("UNet")

    """ Hyperparaqmeters """
    batch_size = 2
    lr = 1e-4  # 0.0001
    num_epochs = 50
    model_path = "UNet/model.h5"
    csv_path = "UNet/data.csv"

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    train_x, train_y = shuffle(train_x, train_y)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    # ds = (1, 2, 3, 4, 5)
    # bs = 2
    # n = len(ds)//bs = 2
    # [1, 2], [3, 4], [1]

    train_steps = (len(train_x)//batch_size)
    valid_steps = (len(valid_x)//batch_size)

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy",
                  optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                          patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )


Train: 1840 - 1840
Valid: 612 - 612
Test: 612 - 612
Epoch 1/50
  3/920 [..............................] - ETA: 18:02 - loss: 0.6822 - dice_coef: 0.0449 - iou: 0.0231 - recall_1: 0.5239 - precision_1: 0.0281 

KeyboardInterrupt: 

In [3]:
from sklearn.model_selection import train_test_split
def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.png")))
    masks = sorted(glob(os.path.join(path, "masks", "*.png")))
    size = int(len(images) * split)
    train_x, valid_x = train_test_split(
        images, test_size=size, random_state=42)
    train_y, valid_y = train_test_split(
        masks, test_size=size, random_state=42)
    train_x, test_x = train_test_split(
        train_x, test_size=size, random_state=42)
    train_y, test_y = train_test_split(
        train_y, test_size=size, random_state=42)
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [4]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.utils import CustomObjectScope
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  # (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    # (256, 256) ----> (256, 256, 1)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x


def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  # (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  # (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    # Find contours of the prediction
    y_pred_gray = cv2.cvtColor(y_pred.astype('uint8'), cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(y_pred_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours in yellow
    y_pred = cv2.drawContours(y_pred, contours, -1, (0, 255, 255), thickness=2)

    temp = y_pred.copy()
    # Make temp image red
    temp[:, :, 0] = 0
    temp[:, :, 1] = 0
    
    overlay = cv2.addWeighted(ori_x, 0.5, temp.astype('uint8'), 0.5, 0)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred, line, overlay], axis=1)
    cv2.imwrite(save_path, cat_images)



if __name__ == "__main__":
    create_dir("UNet_Results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("UNet/model.h5")

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Predicion and metrics values"""
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]
        name = name.split("\\")[-1]

        """Reading the image and mask"""
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """Prediction"""
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"UNet_Results/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(
            y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value,
                     recall_value, precision_value])
        
        
        

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Acurracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving all the results"""
    df = pd.DataFrame(
        SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("UNet/score02.csv")


 11%|█         | 68/612 [00:34<03:30,  2.58it/s] C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 27%|██▋       | 165/612 [01:13<02:58,  2.51it/s]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 27%|██▋       | 167/612 [01:14<02:57,  2.51it/s]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples

Acurracy: 0.99370
F1: 0.76636
Jaccard: 0.67618
Recall: 0.76843
Precision: 0.81118


In [4]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.utils import CustomObjectScope
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os
import numpy as np 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))

    # Convert single-channel image to RGB format
    if x.shape[2] == 1:
        x = cv2.cvtColor(x, cv2.COLOR_GRAY2RGB)

    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  # (1, 256, 256, 3)
    return ori_x, x


def save_result(ori_x, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    # Convert single-channel image to RGB format
    if ori_x.shape[2] == 1:
        ori_x = cv2.cvtColor(ori_x, cv2.COLOR_GRAY2RGB)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0
    temp = y_pred.copy()
    # Make temp image red
    temp[:, :, 0] = 0
    temp[:, :, 1] = 0

    overlay = cv2.addWeighted(ori_x, 0.5, temp.astype('uint8'), 0.5, 0)

    cat_images = np.concatenate([ori_x, line, y_pred, line, overlay], axis=1)
    cv2.imwrite(save_path, cat_images)




if __name__ == "__main__":
    create_dir("UNet_Other_Dataset_Test_Results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("UNet/model.h5")

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/dataset/test"

    """ Prediction and metrics values """
    SCORE = []
    
    test_images = glob(os.path.join(dataset_path, "*.jpg"))

    for x in tqdm(test_images, total=len(test_images)):
        name = os.path.basename(x)

        """ Reading the image """
        ori_x, x = read_image(x)

        """ Prediction """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = os.path.join("UNet_Other_Dataset_Test_Results", name)
        save_result(ori_x, y_pred, save_path)

        """ Flattening the numpy arrays """
        y_pred = y_pred.flatten()

        SCORE.append([name] + y_pred.tolist())

    """ Saving all the results """
    df = pd.DataFrame(SCORE, columns=["Image"] + [f"Pixel_{i}" for i in range(H*W)])
    df.to_csv("results/score02_1.csv", index=False)


100%|██████████| 15/15 [00:09<00:00,  1.62it/s]


In [5]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.utils import CustomObjectScope
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  # (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    # (256, 256) ----> (256, 256, 1)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x


def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3))*255

    ori_y = np.expand_dims(ori_y, axis=-1)  # (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  # (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)*255.0
    temp = y_pred.copy()
    # make temp image red
    temp[:, :, 0] = 0
    temp[:, :, 1] = 0
    
    overlay = cv2.addWeighted(ori_x, 0.5,temp.astype('uint8'), 0.5, 0)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred,line,overlay], axis=1)
    cv2.imwrite(save_path, cat_images)


if __name__ == "__main__":
    create_dir("UNet_Augmented_Results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("UNet/model.h5")

    """ Dataset """
    dataset_path = "../BrainTumorSegmentation/Augmented_Data"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Predicion and metrics values"""
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]
        name = name.split("\\")[-1]

        """Reading the image and mask"""
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """Prediction"""
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"UNet_Augmented_Results/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(
            y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value,
                     recall_value, precision_value])
        
        
        

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Acurracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving all the results"""
    df = pd.DataFrame(
        SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("UNet/score02_UNet_Augment.csv")


  1%|▏         | 52/3676 [00:20<23:11,  2.60it/s]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  3%|▎         | 109/3676 [00:42<22:36,  2.63it/s]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  4%|▍         | 161/3676 [01:02<23:42,  2.47it/s]C:\Users\Kev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted sampl

Acurracy: 0.98422
F1: 0.66047
Jaccard: 0.56882
Recall: 0.69095
Precision: 0.69670
